<a href="https://colab.research.google.com/github/priyanshi23/pythonPostGreSQL/blob/main/datascience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [ ]:
!psql -h mydb.cji2oays8va9.eu-north-1.rds.amazonaws.com -U root_user -d myDB

In [42]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [ ]:
df = pd.read_csv("SouthAfricaDevelopmentIndicators.csv")
df.head()

In [ ]:
#clean table name
#lower case letters
#remove data with spaces
#replace -,/,\\.#,% with _
file = "SouthAfricaDevelopmentIndicators"
clean_tbl_name = file.lower().replace("%","value in percentage").replace(" ","_")
clean_tbl_name

In [ ]:
#clean header names
#lower case letters
#remove data with spaces
#replace -,/,\\.#,% with _

df.columns = [x.upper().replace("%","value in percentage").replace(" ","_") for x in df.columns]

df.columns

In [ ]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'datetime64' : 'timestamp'
}

replacements

In [47]:
col_str = ", ".join("{} {}".format(n,d) for (n,d) in zip(df.columns,df.dtypes.replace(replacements)))
col_str

'LOCATION,"INDICATOR","SUBGROUP","UNIT","DATE","VALUE" varchar'

In [48]:
#open database connection
import psycopg2
conn_string = "host = mydb.cji2oays8va9.eu-north-1.rds.amazonaws.com \
                dbname ='myDB' user='root_user' password='166Fantastic\\!'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print("opened datbase sucessfully")

opened datbase sucessfully


In [49]:
#drop tables with the same name
cursor.execute("drop table if exists South_African_Stats;")

In [50]:
#create table
cursor.execute("create table South_African_Stats(LOCATION varchar,INDICATOR varchar,SUBGROUP varchar,UNIT float,DATE date,VALUE varchar)")


In [51]:
#insert values to table

#save df to csv
import pandas as pd
df = pd.read_csv("SouthAfricaDevelopmentIndicators.csv")
df.to_csv('SouthAfricaDevelopmentIndicators.csv',header=df.columns,index=False,encoding='utf-8')

#open csv file.Save it as a object and upload it to db
my_file = open('SouthAfricaDevelopmentIndicators.csv')
print('file opened in memory')


file opened in memory


In [56]:
#upload to db
cursor.execute("COMMIT")
sql_statement = """
                COPY South_African_Stats FROM STDIN WITH
                 CSV
                 HEADER
                 DELIMITER AS ','
                     """
cursor.copy_expert(sql=sql_statement,file=my_file)
print("file copied to db")

file copied to db


In [ ]:
cursor.execute("grant select on South_African_Stats to public")
conn.commit()
cursor.close()
print("table South_African_Stats imported to db completed")